Projet Python 2A
Arnaud BARRAT • Lucas CUMUNEL • Aloys GALLO


Introduction

Installation

Importation des modules

Tout d'abord, nous importons les bibliothèques permettant de récupérer les données. La bliothèque requests permet d'envoyer des requêtes à des sites web, bs4 permet d'analyser et extraire des données de documents HTML et donc de sites internet, re permet de rechercher des expressions régulières (regex) dans du texte, pandas permet de manipuler des bases de données et rapidfuzz fournit des outils pour comparer des chaînes de caractères.

In [ ]:
import requests
import bs4
import re
import pandas as pd
from rapidfuzz import fuzz
import time
from urllib.parse import quote
from langdetect import detect
from fuzzywuzzy import fuzz 

Récupération des données

On cherche tout d'abord à confectionner une table avec les informations sur le livre et le numéro d'index auquel il correspond. Cette base sera ensuite donnée à l'API pour qu'elle donne un ou plusieurs thème à chaque texte. 
La cellule ci-dessous permet de récupérer le texte de la page "gutindex.all", qui associe à chaque oouvrage un numéro d'index. 

In [ ]:
# URL du fichier d'index des textes
url_liste_textes = "https://www.gutenberg.org/dirs/GUTINDEX.ALL.iso-8859-1.txt"

# Téléchargement du fichier d'index
request_liste_textes = requests.get(url_liste_textes).content
page = bs4.BeautifulSoup(request_liste_textes, "lxml")
body = page.find("body")
index_texte = body.get_text()

Toujours dans l'optique de créer une table avec les informations sur le texte dans une colonne et l'index dans l'autre, on ne souhaite garder que le texte correspondant aux informations et à l'index. 
On utilise pour ce faire les balises de début et de fin de l'index puis on supprime les quelques lignes inutiles qui donnent des informations sur le contenu de l'index après avoir converti le texte en une liste de lignes pour faciliter le traitement.

In [ ]:
# Chercher les indices des marqueurs "<===LISTINGS===>" et "<==End of GUTINDEX.ALL==>"
start_marker = "<===LISTINGS===>"
end_marker = "<==End of GUTINDEX.ALL==>"
start_index = index_texte.find(start_marker)
end_index = index_texte.find(end_marker)

# Extraire le texte entre les marqueurs
texte_extrait = index_texte[start_index + len(start_marker):end_index].strip()
texte_extrait_lignes = texte_extrait.splitlines()

# Filtrer les lignes pertinentes
texte_extrait_lignes_trie = texte_extrait_lignes[10:len(texte_extrait_lignes)-1]
texte_complet = '\n'.join(texte_extrait_lignes_trie)

Il nous faut donc maintenant séparer le texte en deux parties, l'une contenant les informations sur l'oeuvre (titre, auteur, date, langue de l'oeuvre...) et l'autre le numéro d'index.
La cellule ci-dessous, après avoir divisé le texte en oeuvres, créé une liste d'oeuvre avec description (contenant les informations sur l'oeuvre) et index séparés. La première utilisation du regex dans la boucle permet d'extraire l'index tandis que la deuxième permet de retirer l'index et les espaces superflus pour ne garder que le texte pour la colonne "Description". On convertit enfin la liste en dataframe pour la manipuler plus efficacement et on ne garde que les textes en français.

In [ ]:
# Diviser le texte en oeuvres
oeuvres = re.split(r'(?=\n{2,})', texte_complet.strip())

# Extraire les descriptions et indices
# Liste pour stocker les données extraites
data = []
for oeuvre in oeuvres:
    # Trouver l'index dans l'oeuvre
    match_index = re.search(r'(?<=\s\s)([\d]+?[A-Z]?)(?=\n)', oeuvre)
    index = match_index.group(1) if match_index else None

    # Nettoyer le texte de l'oeuvre
    description = re.sub(r'(?<=\s\s)([\d]+?[A-Z]?)(?=\n)', '', oeuvre).strip()

    # Ajouter les données
    data.append({"Description": description, "Index": index})

# Convertir les données en DataFrame
df_livres = pd.DataFrame(data)
df_livres_fr = df_livres[df_livres["Description"].str.contains(r"\[Language: French\]", na=False)]


Afin que l'API nous donne bien des thèmes pour les textes envoyés, nous avons fait le choix, après quellques essais, de nous restreindre à ceux d'auteurs célèbres. Nous avons donc réalisé une liste d'auteurs célèbres des 17
La cellule ci-dessous ne garde que les ouvrages dont l'auteur est dans la liste. 

In [ ]:
# List of French Writers abritrarily defined and chosen in the 17th, 18th and 19th century
auteurs = [
    # 17th century
    "Honoré d'Urfé", "Madeleine de Scudéry", "Paul Scarron", "Jean de La Fontaine",
    "Madame de Lafayette", "Charles Sorel", "Tristan L'Hermite", "François de Salignac de La Mothe-Fénelon",
    "Savinien de Cyrano de Bergerac",
    
    # 18th century
    "Montesquieu", "Voltaire", "Jean-Jacques Rousseau", "Denis Diderot", "Marivaux",
    "Abbé Prévost", "Pierre Choderlos de Laclos", "Beaumarchais", 
    
    # 19th century
    "Honoré de Balzac", "Victor Hugo", "Alexandre Dumas", "Gustave Flaubert", "Émile Zola",
    "Stendhal", "Alfred de Musset", "George Sand", "Jules Verne", "Alphonse Daudet",
    "Théophile Gautier", "Edmond de Goncourt",
    "Joris-Karl Huysmans", "Octave Mirbeau", 
    "Prosper Mérimée", "Eugène Sue", "Charles Nodier",
    "Gaston Leroux", "François-René de Chateaubriand", "Anatole France", "Gustave Flaubert", "Alfred Jarry",
    "Guy de Maupassant", "Romain Rolland", "Alfred Séguin", "Alfred de Vigny", "Paul de Kock"

]

#On créé une expression réulière que signifie "ou" pour l'utiliser ensuite
auteurs_join = "|".join(map(re.escape, auteurs))
# Filtrer les lignes qui contiennent au moins un des auteurs
df_livres_fr_filtré = df_livres_fr[df_livres_fr["Description"].str.contains(auteurs_join, na=False)]
df_livres_fr_filtré.to_csv("livres_fr_triés.csv", index=False, encoding="utf-8")


Récupération des thèmes des livres de ces auteurs et traitement de ces thèmes

Récupération des livres dont on dispose des thèmes 

Il faut maintenant faire correspondre les oeuvres de la base donnée par l'API, c'est-à-dire les oeuvres enrichies des thèmes, avec celles de la base qui contienne leur index de manière à pouvoir aisément récupérer les textes.
La cellule ci-dessous commence par conserver la colonne "Description" pour la mettre dans la nouvelle base car elle sera utile par la suite. Ensuite, on parcourt en parallèle le titre et l'auteur (pour ne pas confondre des oeuvres éponymes). On nettoie les données puis on définit différents manières de faire correspondre le titre à une partie de la description. En effet, sans cela, on perd de nombreux textes en raison de caractères spéciaux ou de sous-titres présents ou non. On définit donc un match comme une situation où un des trois modes de correspondance du titre et de la description est validé et où l'auteur est identique dans les deux bases. On forme une nouvelle base formée des informations sur les textes, de leur thème  et de leur index. On supprime enfin les lignes sans index puisqu'elles ne permettront pas de récupérer de  texte. 

In [ ]:
# Conserver la colonne 'Description' dans la base initiale
base_csv['Description'] = ""
indices = []

# Parcourir les titres et auteurs en parallèle
for title, author in zip(base_csv['Title'], base_csv['Author']):
    # Nettoyage des données : normalisation
    df['Description_clean'] = df['Description'].str.strip().str.lower()
    title_clean = title.strip().lower()
    author_clean = author.strip().lower()

    # Correspondances exactes et approximatives
    df['Exact_Match'] = df['Description_clean'].str.match(rf"^{re.escape(title_clean)}(\s|[.,;!?]|$)", na=False)
    title_words = title_clean.split()
    df['Description_start'] = df['Description_clean'].str.split().str[:len(title_words)].str.join(' ')
    df['Starts_With_Title'] = df['Description_start'] == title_clean
    df['Similarity'] = df['Description_clean'].apply(lambda x: fuzz.ratio(title_clean, x[:len(title_clean)]))
    similarity_threshold = 90
    df['Approx_Match'] = df['Similarity'] > similarity_threshold
    df['Author_Present'] = df['Description_clean'].str.contains(author_clean, na=False)

    # Fusionner les critères
    df['Final_Match'] = (df['Exact_Match'] | df['Starts_With_Title'] | df['Approx_Match']) & df['Author_Present']
    match = df[df['Final_Match']]

    if not match.empty:
        # Ajouter l'index et la description correspondants
        indices.append(match.iloc[0]['Index'])
        base_csv.loc[base_csv['Title'] == title, 'Description'] = match.iloc[0]['Description']
    else:
        indices.append(None)

# Ajouter les indices trouvés à la base
base_csv['Index'] = indices

# Supprimer les lignes sans index trouvé
base_csv_clean = base_csv.dropna(subset=['Index'])

# Sauvegarder la base mise à jour
base_csv_clean.to_csv("base_csv_avec_index.csv", index=False)


On peut enfin réaliser la base de données qui nous intéresse. Elle contient les informations sur les oeuvres, leur thème et leur texte.
Pour ce faire, on itère  sur le titre et l'index pour télécharger le texte du livre (l'URL est standardisée et permet donc cette opération). On vérifie à chaque fois le succès du téléchargement pour éviter les erreurs. On se sert des marqueurs présents dans le texte pour enlever ce qui est superflu. Enfin, on supprime les textes qui ne sont pas en français (nos modèles ne traitent que les textes en français), les lignes pour lesquelles il n'y a pas de texte et les lignes pour lesquelles le type n'est pas le bon (des fichiers audios étaient contenus dans la base). On enregistre enfin la base au format csv pour pouvoir nous en servir plus aisément. 

In [ ]:
# Charger la base nettoyée pour ajouter les textes
base_csv_index = pd.read_csv("base_csv_avec_index.csv")
base_csv_index['Texte'] = ""

# Télécharger les textes des livres
for livre, index in base_csv_index[['Title', 'Index']].itertuples(index=False):
    url = f"https://www.gutenberg.org/cache/epub/{index}/pg{index}-images.html"
    response = requests.get(url)

    if response.status_code == 200:
        soup = bs4.BeautifulSoup(response.text, 'html.parser')
        page_text = soup.get_text()

        start_marker = f"*** START OF THE PROJECT GUTENBERG EBOOK "
        end_marker = f"*** END OF THE PROJECT GUTENBERG EBOOK"
        start_index = page_text.find(start_marker)
        end_index = page_text.find(end_marker)

        if start_index != -1 and end_index != -1:
            texte_extrait = page_text[start_index + len(start_marker):end_index].strip()
            base_csv_index.loc[base_csv_index['Title'] == livre, 'Texte'] = texte_extrait
        else:
            print(f"Marqueurs non trouvés pour {livre} (Index {index}).")
    else:
        print(f"Erreur lors du téléchargement de la page pour {livre} (Index {index}).")

base_csv_index = base_csv_index[base_csv_index["Description"].str.contains(r"\[Language: French\]", na=False)]
base_csv_index = base_csv_index.dropna(subset=['Texte'])
base_csv_index = base_csv_index[~base_csv_index['Description'].str.contains('Audio', na=False)]

# Sauvegarder la base finale
base_csv_index.to_csv("Data/base_csv_final.csv", index=False)
print(base_csv_index.head())
print(base_csv_index.shape[0])


### Deuxième étape de l'extraction des données : Trouver les thèmes des livres disponibles sur Gutenberg

La deuxième étape de l'extraction des données consiste à trouver les thèmes des livres dont on sait qu'on dispose des textes dans Gutenberg. Cependant, comme Gutenberg ne fournit pas cette information, nous avons utilisé l'API **OpenLibrary**, qui est l'une des rares API regroupant les thèmes des livres. 

#### Sélection des auteurs
Pour optimiser l'extraction, nous avons défini une liste d'auteurs à partir des pages Wikipedia recensant les romanciers français, en particulier ceux mentionnés dans la [catégorie des romanciers français par siècle](https://fr.wikipedia.org/wiki/Cat%C3%A9gorie:Romancier_fran%C3%A7ais_par_si%C3%A8cle). Nous avons croisé cette liste avec nos connaissances en prépa B/L pour garder uniquement les auteurs dits "classiques". Cette approche présente trois avantages :
1. **Maximiser les chances de trouver les thèmes des livres** : Les auteurs classiques ont écrit des ouvrages qui sont souvent cités et reconnus, augmentant ainsi les probabilités d’obtenir des informations sur leurs thèmes.
2. **Réduire la taille de la base de données** : En limitant la sélection aux auteurs classiques, nous évitons une requête trop large qui pourrait être difficile à traiter en termes de temps et qui aurait un impact environnemental plus fort.
3. **Faciliter l'application du projet** : En se concentrant sur des auteurs classiques, les livres sont non seulement largement accessibles, mais également facilement retrouvables dans la vie réelle pour des applications pratiques.

Toutes les étapes détaillées du processus sont expliquées dans le notebook [Get_themes.ipynb](#........................#).

#### Nettoyage des données brutes
Nous partons de la liste des livres (fichier `nom_fichier_csv`) pour lesquels nous savons que les textes complets sont disponibles. Après avoir récupéré ces données brutes, nous effectuons un nettoyage pour obtenir un dataframe regroupant les noms des auteurs et les titres des livres. Voici quelques étapes du nettoyage :
1. **Suppression des numéros de tomes apparaissant dans les titres ;**
2. **Suppression des livres en doublon ;** 
3. **Suppression des caractères indésirables (accents mal encodés, symboles inutiles), que nous supprimons pour standardiser les titres.**

#### Requête à l'API OpenLibrary
Une fois les données nettoyées, nous interrogeons l'API **OpenLibrary** pour obtenir les thèmes associés à chaque livre. 

#### Raisons du choix de l'API OpenLibrary
L'API OpenLibrary a été choisie car elle est gratuite, facile d'accès, et ne nécessite pas de clé d'API. De plus, elle contient des informations sur les thèmes des livres, ce qui est essentiel pour l'analyse de nos données.

#### Problème de langue des thèmes
L'API OpenLibrary renvoie les thèmes dans différentes langues, mais jamais en français. C’est pourquoi nous avons utilisé l'API **Lingva** (https://lingva.ml/) pour traduire ces thèmes en français.

#### Raisons du choix de l'API Lingva
L'API **Lingva** a été choisie car elle est gratuite, ne nécessite pas de clé API et permet de traduire les thèmes rapidement. Cette API nous a permis de résoudre le problème de la langue des thèmes obtenus, en les traduisant automatiquement en français.

#### Mapping des thèmes
Une fois les thèmes traduits, nous avons réalisé un **mapping manuel** pour nettoyer les données :
- **Suppression des thèmes non pertinents** : Certains thèmes comme "Langue française" ou "Littérature classique" étaient inutiles pour l'analyse ;
- **Regroupement des thèmes similaires pour créer des catégories plus générales.**

#### Réduction de la taille des données
L'un des principaux défis rencontrés dans ce projet était la taille des données. En effet, nous avons commencé avec un total de 476 livres, mais il était nécessaire de réduire cette quantité pour faciliter les analyses suivantes. Le nettoyage des données a permis d'atteindre un total de 276 livres, mais ce nombre de livre demeurait trop important compte tenu de la puissance de calcul dont nous disposons.

Nous avons donc :
1. **Examiné la fréquence d'apparition des auteurs** : Nous avons limité le nombre de livres par auteur à deux pour éviter d'avoir une base de données trop biaisée.
2. Après cette réduction, nous avons obtenu une base de données finale de **96 livres**.

#### Difficultés rencontrées
Au cours de ces étapes, plusieurs difficultés ont été rencontrées :
1. **Trouver des API pertinentes et gratuites** : trouver des API qui fournissaient les informations dont on avait besoin a été une tâche complexe.
2. **Traductions incomplètes** : L'API ne parvenait pas toujours à traduire l'ensemble des thèmes, même en insérant des pauses via `time.sleep()` avec des durées de plus en plus longues ou en mettant uniquement en langue source 'en' pour spécifier que tous les textes sont initialement en anglais.

 >**Remarque** : Étant donné que la requête de deux API prend chacune plus de 30 minutes, soit un total d'environ une heure, on importe uniquement la base de données après le nettoyage pour voir le résultat.

In [ ]:
df_books = pd.read_csv('final_list')
df_books

Statistiques descriptives des données

Clustering des données

K means

### Clustering des livres : Application de la méthode VBGMM

Pour réaliser un clustering sur les livres, nous avons choisi d'appliquer la méthode **VBGMM (Variational Bayesian Gaussian Mixture Model)**.

#### Pourquoi la méthode VBGMM ?
1. **Estimation du nombre de clusters** :  
   Contrairement à une méthode classique de mélange gaussien (GMM), où le nombre de clusters doit être fixé à l'avance, VBGMM utilise une approche bayésienne variationnelle pour déterminer automatiquement le nombre optimal de clusters en fonction des données.

2. **Gestion des données complexes** :  
   La méthode est particulièrement efficace pour les ensembles de données complexes et de haute dimension, comme ceux impliquant des thèmes littéraires et des métadonnées textuelles.

3. **Réduction des biais** :  
   La pondération bayésienne permet d'éviter le sur-ajustement en pénalisant les clusters inutiles ou redondants.


### Conditions pour utiliser la méthode VBGMM

L'application de la méthode **VBGMM (Variational Bayesian Gaussian Mixture Model)** repose sur certaines conditions et prérequis pour garantir des résultats optimaux et une interprétation valide des clusters.

#### 1. Données numériques dans un espace vectoriel
- Les données d'entrée doivent être représentées sous forme numérique. Nous utilisons donc une matrice Tf-IDF dense, c'est-à-dire dont la majorité des éléments sont différents de zéro.

#### 2. Hypothèse d'une distribution gaussienne
- VBGMM suppose que chaque cluster suit une distribution gaussienne.  


#### 3. Absence de valeurs aberrantes
- Les données ne doivent pas contenir de valeurs aberrantes importantes comme des NaN ou des infinis


In [ ]:
df = pd.read_csv("Data/base_csv_final.csv")
df

In [ ]:

def Tf_Idf (lemmas) :
    
    voc=[]
    for l in lemmas :
        
        voc.extend(l['Lemmes'].tolist())  # Assuming 'Lemmes' column contains the terms
    voc = list(set(voc)) 
    
    vectorizer = TfidfVectorizer(lowercase=False, vocabulary=voc, min_df=2)
    documents = [" ".join(l['Lemmes'].tolist()) for l in lemmas]
    vectorizer.fit(documents)
    vectors=vectorizer.transform(documents)
    """
    vectors=[]
    for l in lemmas :
        X=vectorizer.transform([" ".join(l['Lemmes'].tolist())])
        vectors.append(X)
        """
    return vectors,vectorizer
t_lemmas=[v for k, v in pd.read_parquet('Data/lemmes.parquet', engine='pyarrow').groupby('Id')]
vec,vectorizer=Tf_Idf(t_lemmas)

In [ ]:
# Convertir chaque élément de la liste en tableau dense
vec_dense = [matrix.toarray() for matrix in vec]

# Combiner toutes les matrices en une seule
vec_combined = np.vstack(vec_dense)

# Vérification de la structure des données 
# Vérifier que la moyenne des variances des colonnes n'est pas nulle
print("Moyenne des variances des colonnes :", np.var(vec_combined, axis=0).mean()) 

# Vérifier s'il n'y a pas des NaN
print("Existence de Nan :", np.any(np.isnan(vec_combined)))  

# Vérifier s'il n'y a pas des infinis
print("Existence de valeurs infinis :", np.any(np.isinf(vec_combined)))  

### Vérification des hypothèses et réduction de dimension

Les hypothèses **1** et **3** sont bien vérifiées. Il est maintenant nécessaire de vérifier si les données suivent une **distribution gaussienne**.

Pour ce faire, une **Analyse en Composantes Principales (ACP)** est réalisée. Cette méthode permet de **réduire la dimension des données** tout en préservant le maximum de variance. Elle facilite également la **projection des données** dans un espace de dimension réduite, rendant leur visualisation plus intuitive.

In [ ]:
# Appliquer PCA pour réduire à 2 dimensions
pca = PCA(n_components=2)
reduced_data = pca.fit_transform(vec_combined)

plt.scatter(reduced_data[:, 0], reduced_data[:, 1])
plt.title("PCA Projection of TF-IDF Data")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.show()

### Observation de la répartition des données

Les données semblent être réparties de manière **uniforme autour de l'origine**. Visuellement, cela signifie qu'elles ne présentent pas de **concentration particulière** dans certaines zones de l'espace étudié. 

Cette répartition pourrait suggérer qu'il n'y a pas de **structures sous-jacentes** ou de **tendances marquées**, comme des regroupements naturels de points, qui pourraient indiquer la présence de clusters. 

Cependant, il est important de noter que l'absence de **clusters évidents a priori** ne signifie pas nécessairement qu'il n'en existe pas.

Pour approfondir cette analyse, affichons l'**histogramme de la première et de la seconde composante principale** afin d'examiner la structure des données de manière plus détaillée.


In [ ]:
# Création des sous-graphiques côte à côte
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Histogramme de la première composante principale
sns.histplot(reduced_data[:, 0], kde=True, ax=axes[0])
axes[0].set_title('Histogramme de la première composante principale')

# Histogramme de la seconde composante principale
sns.histplot(reduced_data[:, 1], kde=True, ax=axes[1])
axes[1].set_title('Histogramme de la seconde composante principale')

# Affichage du graphique
plt.tight_layout()
plt.show()

### Analyse des distributions des composantes principales

On observe que les données sont **globalement centrées en 0**, avec des distributions qui semblent suivre des **lois normales**, sans présenter de **valeurs aberrantes** ou d'**asymétries** notables.

Cette première observation suggère que les données pourraient suivre une **distribution normale**. Cependant, pour confirmer cette hypothèse de manière plus rigoureuse, il est nécessaire de procéder à une **évaluation précise**.

Pour ce faire, nous utiliserons les **QQ plots** (quantile-quantile plots), qui permettent de comparer les **quantiles des données** avec ceux d'une distribution normale. Cette méthode aide à valider l'**hypothèse de normalité** des données.

In [ ]:
# Création des sous-graphiques côte à côte
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Q-Q plot de la première composante principale
stats.probplot(reduced_data[:, 0], dist="norm", plot=axes[0])
axes[0].set_title('Q-Q Plot de la première composante principale')

# Q-Q plot de la seconde composante principale
stats.probplot(reduced_data[:, 1], dist="norm", plot=axes[1])
axes[1].set_title('Q-Q Plot de la seconde composante principale')

# Affichage du graphique
plt.tight_layout()
plt.show()

### Validation de la normalité avec les QQ plots

Hormis pour les **valeurs aux extrémités**, les points des **QQ plots** suivent de manière précise la **diagonale** correspondant à la distribution théorique de la **loi normale**. 

Cette observation indique que les **quantiles des données** s'alignent étroitement avec ceux d'une distribution normale, confirmant ainsi la normalité des données.

Ces résultats permettent d'affirmer, avec beaucoup de confiance, que la **distribution des données suit une loi normale**.


In [ ]:
# Kolmogorov-Smirnov test for normality
stat1, p1 = kstest(reduced_data[:, 0], 'norm', args=(reduced_data[:, 0].mean(), reduced_data[:, 0].std()))
stat2, p2 = kstest(reduced_data[:, 1], 'norm', args=(reduced_data[:, 1].mean(), reduced_data[:, 1].std()))
print(f'Kolmogorov-Smirnov Test for First Principal Component: Statistics={stat1}, p-value={p1}')
print(f'Kolmogorov-Smirnov Test for Second Principal Component: Statistics={stat2}, p-value={p2}')


Le **test de Kolmogorov-Smirnov** (KS) est un test statistique non paramétrique utilisé pour comparer la distribution empirique des données avec une distribution théorique donnée.

## Hypothèses du test

- **Hypothèse nulle (H₀)** : Les données suivent la distribution théorique spécifiée.  


- **Hypothèse alternative (H₁)** : Les données ne suivent pas la distribution théorique spécifiée.  

## Analyse de la p-valeur

Les **p-valeurs** sont ici très élevées. Cela signifie que nous ne rejetons pas l'hypothèse nulle et que nous pouvons conclure que les données suivent bien la distribution théorique avec une grande certitude.

## Conclusion

L'hypothèse 3 est ainsi vérifiée. Nous pouvons désormais appliquer la méthode **VBGMM** (Variational Bayesian Gaussian Mixture Model) pour réaliser des clusters.

In [ ]:
# Create a VBGMM model
vbgmm = BayesianGaussianMixture(n_components=6, covariance_type='full')
    
# Fit the model to the data
vbgmm.fit(reduced_data)
    
# Predict the cluster for each sample
labels = vbgmm.predict(reduced_data)
    
df['Cluster'] = labels

# Plot the clusters
plt.scatter(reduced_data[:, 0], reduced_data[:, 1], c=labels, cmap='viridis')
plt.title('Clusters of Books')
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.show()


In [ ]:
# Create a VBGMM model
vbgmm = BayesianGaussianMixture(n_components=6, covariance_type='full')
    
# Fit the model to the data
vbgmm.fit(reduced_data)
    
# Predict the cluster for each sample
labels = vbgmm.predict(reduced_data)
    
df['Cluster'] = labels

# Plot the clusters
plt.scatter(reduced_data[:, 0], reduced_data[:, 1], c=labels, cmap='viridis')
plt.title('Clusters of Books')
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.show()

### Variabilité des clusters lors des exécutions répétées

On remarque qu'en appliquant plusieurs fois le même algorithme, les **clusters** obtenus sont modifiés. Cela est probablement dû à l'**initialisation aléatoire** des paramètres du modèle, qui peut influencer les résultats finaux.

Pour garantir la **reproductibilité** de l'analyse et éviter les variations liées à cette initialisation, nous allons fixer la **graine aléatoire** (*seed*). Cela permettra d'obtenir des résultats cohérents à chaque exécution de l'algorithme.

Poursuivons donc l'analyse en fixant cette graine.


In [ ]:
np.random.seed()

# Create a VBGMM model
vbgmm = BayesianGaussianMixture(n_components=6, covariance_type='full')
    
# Fit the model to the data
vbgmm.fit(reduced_data)
    
# Predict the cluster for each sample
labels = vbgmm.predict(reduced_data)
    
df['Cluster'] = labels

# Plot the clusters
plt.scatter(reduced_data[:, 0], reduced_data[:, 1], c=labels, cmap='viridis')
plt.title('Clusters of Books')
plt.xlabel('First Principal Component')
plt.ylabel('Second Principal Component')
plt.show()

### Résultat de l'algorithme VBGMM

L'algorithme **VBGMM (Variational Bayesian Gaussian Mixture Model)** a permis de distinguer **trois clusters** dans les données. Ces clusters représentent des regroupements naturels, identifiés à partir des caractéristiques des points dans l'espace étudié.

Avant d'analyser ces clusters en détail pour mieux comprendre leurs spécificités et les thématiques qu'ils regroupent, étudions la qualité des clusters.

In [ ]:
# Calculer les centroïdes des clusters
def compute_centroids(X, labels):
    centroids = []
    for cluster in np.unique(labels):
        cluster_points = X[labels == cluster]
        centroid = cluster_points.mean(axis=0)
        centroids.append(centroid)
    return np.array(centroids)

# Calcul de la distance intra-cluster (moyenne des distances aux centroïdes)
def intra_cluster_distance(X, labels, centroids):
    intra_distances = []
    for cluster in np.unique(labels):
        cluster_points = X[labels == cluster]
        centroid = centroids[cluster]
        distance = np.mean(np.linalg.norm(cluster_points - centroid, axis=1))
        intra_distances.append(distance)
    return np.mean(intra_distances)

# Calcul de la distance inter-cluster (moyenne des distances entre clusters)
def inter_cluster_distance(X, labels, centroids):
    inter_distances = []
    for i, centroid_i in enumerate(centroids):
        for j, centroid_j in enumerate(centroids):
            if i != j:
                dist = np.linalg.norm(centroid_i - centroid_j)
                inter_distances.append(dist)
    return np.mean(inter_distances)


# Calcul des centroïdes
centroids = compute_centroids(df, labels)

# Calcul de la distance intra-cluster
intra_dist = intra_cluster_distance(df, labels, centroids)

# Calcul de la distance inter-cluster
inter_dist = inter_cluster_distance(df, labels, centroids)

print(f"Distance intra-cluster : {intra_dist}")
print(f"Distance inter-cluster : {inter_dist}")


### Évaluation de la qualité du modèle

L'évaluation de la qualité du modèle en utilisant les distances inter et intra-clusters n'est pas possible en raison de limitations computationnelles, entraînant l'erreur suivante : **"Maximum call stack size exceeded"**. 

Pour contourner cette contrainte, nous avons décidé d'évaluer les clusters à l'aide du **score de silhouette**. 

Le **score de silhouette** mesure la cohésion et la séparation des clusters :  
- Il calcule la **différence entre la distance moyenne d'un point avec les autres points de son cluster (cohésion)** et la **distance moyenne avec les points du cluster voisin le plus proche (séparation)**.
- Un score de silhouette proche de 1 indique que les clusters sont bien définis, c'est-à-dire qu'ils ont une forte cohésion et une bonne séparation. 


In [ ]:
vbgmm = BayesianGaussianMixture(n_components=3, random_state=42)
vbgmm.fit(reduced_data)
labels = vbgmm.predict(reduced_data)

# Calcul du silhouette score
f"Silhouette Score : {silhouette_score(reduced_data, labels)}"

### Interprétation du score de silhouette

Un score de silhouette de **0,42** indique que les clusters formés par l'algorithme sont **relativement pertinents**. Cela suggère que les points d'un même cluster sont modérément proches les uns des autres, tandis qu'ils sont bien séparés des autres clusters.

### Limitation et alternative

Pour améliorer la prise en compte des thèmes, une autre approche aurait été d'encoder les thèmes dans une matrice **one-hot** et de la **concaténer avec la matrice TF-IDF**. Cependant, cette méthode présente un inconvénient majeur :  
- **Modification de la distribution des données** : Une telle transformation affecte la structure statistique des données, qui ne suivraient plus une loi gaussienne.  
- Cela rendrait l'algorithme **VBGMM caduc**, car celui-ci repose sur l'hypothèse de normalité des données.

### Comparaison des thèmes des clusters

Pour répondre à la problématique et analyser les différences entre les clusters, nous comparons les thèmes associés à chaque cluster. Une représentation sous forme de **heatmap** est utilisée pour visualiser la fréquence des thèmes dans chaque cluster. Cette approche permet de mettre en lumière les caractéristiques dominantes de chaque cluster et d'explorer les relations entre thèmes et regroupements.


In [ ]:
print("Nombre de livres dans le cluster 0 : ", df['Cluster'].value_counts()[0])
print("Nombre de livres dans le cluster 1 : ", df['Cluster'].value_counts()[1])
print("Nombre de livres dans le cluster 2 : ", df['Cluster'].value_counts()[2])


df['Themes'] = df['Themes'].apply(lambda x: x.split(', ') if isinstance(x, str) else x)

df_exploded = df.explode('Themes')
theme_counts = df_exploded.groupby(['Cluster', 'Themes']).size().unstack(fill_value=0)

# Créer la heatmap pour visualiser les comptes de thèmes par cluster
plt.figure(figsize=(10, 6))
sns.heatmap(theme_counts, annot=True, cmap='Blues', fmt='d', cbar=True)

# Ajouter des titres et labels
plt.title('Nombre d\'apparitions de chaque thème par cluster', fontsize=14)
plt.xlabel('Thèmes', fontsize=12)
plt.ylabel('Clusters', fontsize=12)

# Afficher la heatmap
plt.tight_layout()
plt.show()


## Analyse des clusters

### Cluster 0 et Cluster 2 : Similarités et différences
On remarque que les clusters 0 et 2 sont relativement similaires, principalement en raison de la forte occurrence des thèmes tels que **décadence**, **roman**, **société et politique**, **fiction historique**, **pouvoir** et **coutumes**. Ces thèmes suggèrent que les livres dans ces deux clusters partagent des préoccupations liées aux dynamiques sociales et politiques, explorant les relations humaines, les conflits de pouvoir, ainsi que les dilemmes moraux dans des contextes historiques.

**Cluster 0** se distingue par une présence supplémentaire des thèmes **passions** et **ambitions**. Cela suggère que ce cluster regroupe des romans qui ne se contentent pas de décrire des contextes sociaux ou politiques, mais qui mettent également l'accent sur les émotions humaines et les aspirations personnelles, rendant ces récits plus intenses et dramatiques. Ces livres explorent les enjeux individuels au sein de sociétés marquées par des tensions sociales et politiques, avec une forte dimension historique.

**Cluster 2**, bien qu'il partage des thèmes similaires, semble davantage mettre l'accent sur des aspects **institutionnels** et **sociaux**, tels que la **culture**, les **mœurs**, et **l'éducation**. Cela suggère que les livres du cluster 2 se concentrent peut-être plus sur les structures sociales et les valeurs culturelles que sur les dynamiques émotionnelles et les conflits personnels. Ces romans abordent probablement des questionnements sur les normes et les pratiques sociales dans un cadre historique ou social donné, sans nécessairement explorer les passions ou les ambitions des individus.

### Cluster 1 : Romans d'aventure et de voyage
Le **cluster 1** semble se différencier nettement des autres en mettant l'accent sur les **romans d'aventure** et de **voyage**, souvent caractérisés par une **dimension fantastique** ou **merveilleuse**. Les thèmes présents dans ce cluster suggèrent des récits qui, tout en étant parfois ancrés dans une certaine réalité géographique, s'éloignent dans une certaine mesure des préoccupations sociales ou politiques pour se concentrer sur des aventures extraordinaires, des explorations de nouveaux mondes ou des quêtes héroïques. Ces livres semblent inviter le lecteur à s'évader, en découvrant des univers lointains et fascinants, souvent empreints de mystère ou de fantastique.

Conclusions

Pistes d'amélioration

NameError: name 'Projet' is not defined